### Imports 👽

In [1]:
import os
import sys
import pandas as pd
import torch

sys.path.append(os.path.abspath(os.path.join('..', 'src')))
from data_classes.architecture import NeuralNetworkArchitecture
from data_classes.enums import OptimizationMethod
from data_classes.experiment import Experiment, SamplingMethod
from data_classes.scenario import Scenario, ScenarioSettings
from data_classes.training_data import InputData
from data_classes.training_config import AdamTrainingConfig, TrainingSettings
from models import SequentialNeuralNetwork

### Setup Scenario and Training Settings 🧪

In [2]:
experiments = [
    Experiment(
        SAMPLING_METHOD=SamplingMethod.SOBOL,
        SCENARIO=Scenario.SUM_SINES
    ),
    Experiment(
        SAMPLING_METHOD=SamplingMethod.MC,
        SCENARIO=Scenario.SUM_SINES
    ),
    Experiment(
        SAMPLING_METHOD=SamplingMethod.SOBOL,
        SCENARIO=Scenario.PROJECTILE
    ),
    Experiment(
        SAMPLING_METHOD=SamplingMethod.MC,
        SCENARIO=Scenario.PROJECTILE
    )
]

num_epochs_list = [100, 500, 1000, 1500, 2000]
widths = [6,12,24]
depths = [4,8,12,16,20]
learning_rates = [0.01,0.001]
lambdas = [1.0e-04,1.0e-05,1.0e-06,1e-07]

In [3]:
def get_training_settings(num_epochs: int, scenario_settings: ScenarioSettings):
    training_settings = []
    for training_set_size in scenario_settings.TRAINING_SET_SIZES:
        for width in widths:
            for depth in depths:
                for learning_rate in learning_rates:
                    for lambda_ in lambdas:
                            nn_arch = NeuralNetworkArchitecture(
                                INPUT_DIM=scenario_settings.INPUT_DIM,
                                OUTPUT_DIM=scenario_settings.OUTPUT_DIM,
                                NUM_HIDDEN_LAYERS=width,
                                DEPTH=depth,
                                ACTIVATION_FUNCTION=torch.nn.Sigmoid
                            )
                            training_config = AdamTrainingConfig(
                                OPTIMIZER=OptimizationMethod.ADAM,
                                LEARNING_RATE=learning_rate,
                                REG_PARAM=lambda_,
                                NUM_EPOCHS=num_epochs
                            )

                            training_settings.append(TrainingSettings(
                                nn_architecture=nn_arch,
                                training_config=training_config,
                                training_set_size=training_set_size
                            ))
    return training_settings

### Run Training 🚀

In [4]:
from IPython.display import clear_output

finished_scenarios = []

for experiment in experiments:
    scenario_settings = ScenarioSettings(experiment.SCENARIO)

    input_data = InputData(scenario_settings.DATA_PATH)
    for num_epochs in num_epochs_list:
        finished_scenarios.append(f'{experiment.SCENARIO.value}, {experiment.SAMPLING_METHOD.value}, {num_epochs} epochs')
        training_settings = get_training_settings(num_epochs, scenario_settings)
        training_results_list = []

        output_dir = os.path.abspath(os.path.join('..', 'data', experiment.SCENARIO.value, 'output'))
        os.makedirs(output_dir, exist_ok=True)
        csv_path = os.path.join(output_dir, f'{experiment.SAMPLING_METHOD.value}_{num_epochs}epochs_results.csv')

        for ts in training_settings:
            print('Started:')
            for sc in finished_scenarios:
                print(sc)
            print()
            print('Starting Experiment:')
            print(f'Scenario:                {experiment.SCENARIO.value}')
            print(f'Sampling Method:         {experiment.SAMPLING_METHOD.value}')
            print(f'Number of Epochs:        {num_epochs}')
            print(f'Training Set Size:       {ts.training_set_size}')
            print(f'Number of Hidden Layers: {ts.nn_architecture.NUM_HIDDEN_LAYERS}')
            print(f'Depth:                   {ts.nn_architecture.DEPTH}')
            print(f'Learning-Rate:           {ts.training_config.LEARNING_RATE}')
            print(f'Regression Parameter:    {ts.training_config.REG_PARAM}')
            clear_output(wait=True)
            nn = SequentialNeuralNetwork(
                net_arch=ts.nn_architecture
            )
            training_data = input_data.get_training_and_test_data(
                sampling_method=experiment.SAMPLING_METHOD,
                training_set_size=ts.training_set_size
            )
            nn.train(settings=ts.training_config, data=training_data)
            training_results_list.append(nn.training_results)
            df_results = pd.DataFrame(training_results_list)

            df_results.sort_values('test_error', inplace=True)
            df_results.reset_index(drop=True, inplace=True)
            df_results.to_csv(csv_path, index=False)

KeyboardInterrupt: 